In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as mlt

pd.set_option('max_columns',1000)
pd.set_option('max_row',300)

In [3]:
shang = pd.read_csv('商用车数据.csv',encoding='gbk')

In [4]:
shang_des = shang.describe().T
nouse = shang_des[shang_des['min']==shang_des['max']].index
print(len(nouse))
shang_2 = shang.drop(columns=nouse)
print(shang_2.shape)

14
(236002, 32)


In [5]:
#时间格式转化并diff
shang_2['time'] = shang_2['时间'].apply(lambda x:x[0:4]+'-'+x[5:7]+'-'+x[8:10]+' '+x[11:13]+':'+x[14:16]+':'+x[17:19])
shang_2['time'] = shang_2['time'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
shang_2['time_diff'] = shang_2['time'].diff()
shang_2['time_diff'] = shang_2['time_diff'].apply(lambda x:x.total_seconds())
shang_2['time_diff'] = shang_2['time_diff'].fillna(5.0)
shang_2['time_cumsum'] = shang_2['time_diff'].cumsum()

In [6]:
#时间diff大于180s设置起始点，并按顺序标记每一段slice
shang_2['startpoint'] = 0
shang_2.loc[0,'startpoint'] = 1
shang_2.loc[shang_2['time_diff']>180,'startpoint'] = 1
shang_2['slice'] = shang_2.startpoint.cumsum()

In [7]:
#有些片段总油耗不变，进行删除处理,并将slice重新排序
shang_3 = shang_2.copy()
for i in range(1,shang_2.slice.max()+1):
    slice_ = shang_3[shang_3['slice']==i]
    if slice_['总油耗'].min() == slice_['总油耗'].max():
        shang_3 = shang_3[shang_3['slice']!=i]
shang_3.slice = shang_3.startpoint.cumsum()

print('原先共有片段数：',len(set(shang_2.slice)))
print('删除后有片段数：',len(set(shang_3.slice)))

原先共有片段数： 235
删除后有片段数： 192


In [45]:
#直接corr
corr_direct = pd.DataFrame(shang_3.corr()['总油耗'])
corr_direct = corr_direct.fillna(0.0)
corr_direct

,总油耗
GPS经度,0.153246
GPS纬度,0.151318
GPS车速,-0.005968
GPS里程,0.116502
GPS方向,0.000314
绝对增压压力,0.002101
电瓶电压,-0.115517
油门,-0.001647
单程油耗,0.078787
车速,-0.000161


In [43]:
#192段行程平均corr；
corr = pd.DataFrame(shang_3[shang_3['slice']==1].corr()['总油耗'])
corr = corr.fillna(0.0)
for i in range(2,shang_3.slice.max()+1):
    
    slice_ = shang_3[shang_3['slice']==i]
    pd_slice = pd.DataFrame(slice_.corr()['总油耗'])
    pd_slice = pd_slice.fillna(0.0)
    corr += abs(pd_slice)
corr/shang_3.slice.max()

,总油耗
GPS经度,0.485875
GPS纬度,0.526071
GPS车速,0.208597
GPS里程,0.683714
GPS方向,0.218307
绝对增压压力,0.164290
电瓶电压,0.214404
油门,0.148375
单程油耗,0.987866
车速,0.210982


In [44]:
corr_df[['mean','num']]

,mean,num
GPS经度,0.488419,1
GPS纬度,0.528825,1
GPS车速,0.215326,6
GPS里程,0.713441,8
GPS方向,0.225349,6
绝对增压压力,0.166898,3
电瓶电压,0.217974,1
油门,0.152342,5
单程油耗,0.993038,1
车速,0.216624,5


In [49]:
corr_df[['mean','num']].sort_values(by=['mean'],ascending=False)

,mean,num
总油耗,1.000000,0
单程油耗,0.993038,1
ECU里程,0.931548,9
time_cumsum,0.922805,1
发动机运行总时间,0.914977,1
GPS里程,0.713441,8
GPS纬度,0.528825,1
GPS经度,0.488419,1
机油温度,0.481838,1
水温,0.421678,1


In [36]:
#去除片段中某些参数不变情况下的平均corr
list_columns = list(shang_3.columns)
nouse_columns = ['总油耗','ECU的故障状态','限制模式','ECU和终端的通信状态','ECU锁车状态','time_diff','startpoint','slice','时间','time']
for i in nouse_columns:
    list_columns.remove(i)

corr_df = pd.DataFrame()
for i in range(1,shang_3.slice.max()+1):
    slice_ = shang_3[shang_3['slice']==i]
    pd_slice = pd.DataFrame(slice_.corr()['总油耗'])
    pd_slice = pd_slice.fillna(0.0)
    corr_df['{}'.format(i)] = pd_slice['总油耗']
    
#计有几个0
corr_df['num'] = 0
for i in list_columns:
    corr_df.loc[i,'num'] = sum(corr_df.loc[i,:]==0)
    
#只计算变量发生变化后的corr均值
corr_df['mean'] = 0
for i in range(1,shang_3.slice.max()+1):
    corr_df['mean'] += abs(corr_df['{}'.format(i)])
corr_df['mean'] =  corr_df['mean']/(shang_3.slice.max()-corr_df['num'])